In [1]:
!pip install numpy pandas keras scikit-learn statsmodels boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 146.8 MB/s eta 0:00:00


In [5]:
import boto3
import pandas as pd

# Set up the S3 client
s3 = boto3.client('s3')

# Define the S3 bucket and the key (file path) for the file you want to download
bucket_name = 'crypto-prediction-project'
file_key = 'data/Bitcoin Historical Data.csv'
local_file_name = 'bitcoin_data.csv'

# Download data from the S3 bucket to the local system
s3.download_file(bucket_name, file_key, local_file_name)

# Load the data into a DataFrame
df = pd.read_csv(local_file_name)

# Display the first few rows of the DataFrame
print(df.head())


         Date     Price      Open      High       Low     Vol. Change %
0  03/24/2024  67,211.9  64,036.5  67,587.8  63,812.9   65.59K    4.96%
1  03/23/2024  64,037.8  63,785.6  65,972.4  63,074.9   35.11K    0.40%
2  03/22/2024  63,785.5  65,501.5  66,633.3  62,328.3   72.43K   -2.62%
3  03/21/2024  65,503.8  67,860.0  68,161.7  64,616.1   75.26K   -3.46%
4  03/20/2024  67,854.0  62,046.8  68,029.5  60,850.9  133.53K    9.35%


In [7]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 24.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 121.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 105.0 MB/s eta 0:00:00


In [10]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Load data
# Assuming 'df' is already loaded with your dataset (e.g., from S3 or CSV)

# Remove commas from the 'Price' column and convert to float
df['Price'] = df['Price'].replace({',': ''}, regex=True).astype(float)

# Data preprocessing: Scale the features and labels
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Price']].values)

# Train-test split
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

# Prepare data for LSTM (LSTM expects 3D input: [samples, timesteps, features])
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])  # Features
        y.append(data[i + time_step, 0])      # Label
    return np.array(X), np.array(y)

# Reshape data for LSTM
time_step = 60  # Use previous 60 days to predict the next day
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape inputs to be 3D [samples, time steps, features] for LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Create LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))  # Output layer

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Save the model
model.save('bitcoin_lstm_model.h5')


Epoch 1/10


2025-04-27 05:03:37.539011: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


124/124 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - loss: 0.0186
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - loss: 5.3531e-04
Epoch 3/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - loss: 4.6144e-04
Epoch 4/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 3.6696e-04
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - loss: 5.3785e-04
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 3.5812e-04
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5s 43ms/step - loss: 3.1768e-04
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - loss: 4.8225e-04
Epoch 9/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - loss: 2.6213e-04
Epoch 10/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - loss: 2.4186e-04


In [14]:
import statsmodels.api as sm

# Fit an ARIMA model on 'Price' series
model = sm.tsa.ARIMA(df['Price'], order=(1, 1, 1))  # (p,d,q) can be tuned
model_fit = model.fit()

# Save the model
model_fit.save('bitcoin_arima_model.pkl')

# Forecast the next 5 values
forecast = model_fit.forecast(steps=5)
print(forecast)

4999    0.1
5000    0.1
5001    0.1
5002    0.1
5003    0.1
Name: predicted_mean, dtype: float64


In [15]:
import requests
import boto3

def lambda_handler(event, context):
    # Get live data from an API (e.g., CoinGecko)
    response = requests.get('https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd')
    price = response.json()['bitcoin']['usd']
    
    # Call SageMaker endpoint for prediction
    sagemaker_client = boto3.client('sagemaker-runtime')
    endpoint_name = 'your-endpoint-name'
    payload = f'{{"price": {price}}}'
    
    response = sagemaker_client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=payload)
    
    prediction = response['Body'].read().decode('utf-8')
    
    return {
        'statusCode': 200,
        'body': prediction
    }


In [16]:
import requests
import boto3
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import statsmodels.api as sm

def lambda_handler(event, context):
    # Get live data from an API (e.g., CoinGecko)
    response = requests.get('https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd')
    price = response.json()['bitcoin']['usd']
    
    # Initialize the prediction output dictionary
    prediction_output = {}

    # Load LSTM Model and make prediction
    try:
        lstm_model = load_model('bitcoin_lstm_model.h5')
        
        # Preprocess data for LSTM (rescaling)
        scaler = MinMaxScaler(feature_range=(0, 1))
        price_scaled = scaler.fit_transform(np.array([[price]]))

        # Reshape data to match LSTM input (time_step of 60)
        time_step = 60
        X_input = np.reshape(price_scaled, (1, 1, 1))  # Reshape for single timestep

        # LSTM prediction
        lstm_prediction = lstm_model.predict(X_input)
        lstm_prediction = scaler.inverse_transform(lstm_prediction)  # Rescale back
        prediction_output['LSTM_Prediction'] = lstm_prediction[0][0]
    except Exception as e:
        prediction_output['LSTM_Error'] = str(e)
    
    # Load ARIMA Model and make prediction
    try:
        arima_model = sm.tsa.ARIMA.load('bitcoin_arima_model.pkl')
        # Use the previous price for ARIMA prediction
        df = pd.DataFrame({'Price': [price]})
        arima_model_fit = arima_model.fit()
        forecast = arima_model_fit.forecast(steps=5)
        prediction_output['ARIMA_Forecast'] = forecast
    except Exception as e:
        prediction_output['ARIMA_Error'] = str(e)

    return {
        'statusCode': 200,
        'body': prediction_output
    }


In [17]:
import boto3

s3 = boto3.client('s3')

# Upload LSTM model
s3.upload_file('bitcoin_lstm_model.h5', 'crypto-prediction-project', 'models/bitcoin_lstm_model.h5')

# Upload ARIMA model
s3.upload_file('bitcoin_arima_model.pkl', 'crypto-prediction-project', 'models/bitcoin_arima_model.pkl')


In [19]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()  # Get the SageMaker execution role
sagemaker_client = boto3.client('sagemaker')

model_name = 'bitcoin-lstm-model'
model_data = 's3://crypto-prediction-project/models/bitcoin_lstm_model.h5'  # S3 path of the model

# Specify the pre-built TensorFlow container image
tensorflow_inference_image = '763104351884.dkr.ecr.us-west-1.amazonaws.com/tensorflow-inference:2.3.0-cpu'

response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': tensorflow_inference_image,  # Use TensorFlow inference image
        'ModelDataUrl': model_data,
    },
    ExecutionRoleArn=role
)

print(response)


{'ModelArn': 'arn:aws:sagemaker:us-west-1:515966506866:model/bitcoin-lstm-model', 'ResponseMetadata': {'RequestId': '080b7af2-892b-461e-9623-22408e3dd166', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '080b7af2-892b-461e-9623-22408e3dd166', 'content-type': 'application/x-amz-json-1.1', 'content-length': '80', 'date': 'Sun, 27 Apr 2025 06:11:41 GMT'}, 'RetryAttempts': 0}}


In [20]:
# Create an endpoint configuration
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName='bitcoin-lstm-endpoint-config',
    ProductionVariants=[
        {
            'InstanceType': 'ml.m5.large',  # Choose instance type
            'InitialInstanceCount': 1,
            'ModelName': model_name,
            'VariantName': 'AllTraffic'
        }
    ]
)

# Create an endpoint
endpoint_response = sagemaker_client.create_endpoint(
    EndpointName='bitcoin-lstm-endpoint',
    EndpointConfigName='bitcoin-lstm-endpoint-config'
)

print(endpoint_response)


{'EndpointArn': 'arn:aws:sagemaker:us-west-1:515966506866:endpoint/bitcoin-lstm-endpoint', 'ResponseMetadata': {'RequestId': 'e59d8e41-9f27-4cc1-a08a-fcd4ff023f48', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e59d8e41-9f27-4cc1-a08a-fcd4ff023f48', 'content-type': 'application/x-amz-json-1.1', 'content-length': '89', 'date': 'Sun, 27 Apr 2025 06:12:08 GMT'}, 'RetryAttempts': 0}}


In [22]:
import boto3

# Create a SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all endpoints
response = sagemaker_client.list_endpoints()

# Print the available endpoints
for endpoint in response['Endpoints']:
    print(endpoint['EndpointName'])


bitcoin-lstm-endpoint


In [23]:
import boto3

# Create a SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime')

# Specify the endpoint name and model details
endpoint_name = "bitcoin-lstm-endpoint" 


In [24]:
response = sagemaker_client.describe_endpoint(EndpointName="bitcoin-lstm-endpoint")
print(response['EndpointStatus'])  # Should show 'InService' if the endpoint is healthy


Creating


In [32]:
import tarfile

# Define the model file and the target tar.gz file
model_file = 'bitcoin_lstm_model.h5'
tar_gz_file = 'bitcoin_lstm_model.tar.gz'

# Create a tar.gz archive
with tarfile.open(tar_gz_file, 'w:gz') as tar:
    tar.add(model_file, arcname='bitcoin_lstm_model.h5')

print(f'Model file {model_file} compressed into {tar_gz_file}')


Model file bitcoin_lstm_model.h5 compressed into bitcoin_lstm_model.tar.gz


In [33]:
import boto3

# Define your S3 bucket and object key
bucket_name = 'crypto-prediction-project'
object_key = 'models/bitcoin_lstm_model.tar.gz'

# Create an S3 client
s3_client = boto3.client('s3')

# Upload the file to S3
s3_client.upload_file(tar_gz_file, bucket_name, object_key)

print(f'File uploaded to s3://{bucket_name}/{object_key}')


File uploaded to s3://crypto-prediction-project/models/bitcoin_lstm_model.tar.gz


In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlowModel

# Set up SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

# Specify the S3 path to your .tar.gz model file
model_uri = 's3://crypto-prediction-project/models/bitcoin_lstm_model.tar.gz'

# Create a TensorFlow model from the .tar.gz file
tensorflow_model = TensorFlowModel(model_data=model_uri,
                                   role=role,
                                   framework_version='2.3.0',  # Specify the TensorFlow version
                                   sagemaker_session=sagemaker_session)

# Deploy the model to an endpoint
endpoint = tensorflow_model.deploy(instance_type='ml.m5.large', initial_instance_count=1)

print(f'Endpoint created: {endpoint}')


In [ ]:
import json

# Prepare input data
input_data = json.dumps({'price': 50000})  # Example input data

# Call the endpoint
response = sagemaker_runtime.invoke_endpoint(EndpointName=endpoint.endpoint_name,
                                             ContentType='application/json',
                                             Body=input_data)

# Parse the response
result = json.loads(response['Body'].read().decode())
print(f'Prediction result: {result}')
